In [1]:
import pandas as pd
import numpy as np 

# Reading the csv file

In [2]:
df = pd.read_csv("20210103_hundenamen.csv")

In [3]:
pd.set_option('display.max_rows', 9000)

In [4]:
df

,HUNDENAME,GEBURTSJAHR_HUND,GESCHLECHT_HUND
0,Ituma,2011,w
1,"""Bo"" Bendy of Treegarden",2020,m
2,"""Bobby"" Lord Sinclair",2009,m
3,"""Buddy"" Fortheringhay's J.",2011,m
4,"""Fly"" Showring i fly for you",2015,w
5,"""Pino"" Ami du soleil",2010,m
6,"""Zappalla II"" Kora v. Tüfibach",2011,w
7,A great Dream Kajsa of Moss-La,2019,w
8,A-Diana,2006,w
9,AISHA von der Mark,2020,w


In [5]:
df.value_counts()

HUNDENAME                       GEBURTSJAHR_HUND  GESCHLECHT_HUND
Luna                            2020              w                  13
                                2018              w                  10
Lola                            2012              w                   9
Luna                            2013              w                   9
Nala                            2020              w                   9
Luna                            2016              w                   8
                                2019              w                   7
Molly                           2020              w                   6
Luna                            2011              w                   6
Mia                             2018              w                   6
Nina                            2014              w                   6
Rocky                           2019              m                   5
Mila                            2020              w                   

It seems like there are many rows that don't have a known value (unbekannt).
These can be dropped

In [6]:
df[df['HUNDENAME']=="unbekannt"]

,HUNDENAME,GEBURTSJAHR_HUND,GESCHLECHT_HUND
8569,unbekannt,2010,w
8570,unbekannt,2011,m
8571,unbekannt,2018,m
8572,unbekannt,2018,m
8573,unbekannt,2017,m


In [7]:
df = df.drop([8569, 8570, 8571, 8572, 8573])

The distance for the same name should be calculated only once. 
Get only unique dog names.

In [8]:
unique_names = set(df['HUNDENAME'])

# The easiest option

Importing an existing library, which is optimized for this calculation

In [9]:
from Levenshtein import distance as levenshtein_distance

In [10]:
target = "Luca"

In [11]:
results = [n for n in unique_names if levenshtein_distance(n, target) == 1]

In [12]:
results

['Lucy',
 'Luce',
 'Lua',
 'Lula',
 'Yuca',
 'Lucas',
 'Luba',
 'Lucia',
 'Lupa',
 'Luma',
 'Cuca',
 'Luna']

# Implementing an independent solution

Based on https://en.wikipedia.org/wiki/Wagner%E2%80%93Fischer_algorithm

In [15]:
def get_distance(str1: str, str2: str) -> int:
    '''
    Compares two strings and returns Levenshtein distance between them.
    Levenshtein distance between two strings is the minimum number of edits (per single character)
    that are required to change the first string into the second string (insertion, deletion and substitution).
    Param: str1 = a string with a length of more than 0, str2  = a string with a length of more than 0
    Return: An integer (Levenshtein distance)
    
    Edge cases are not considered here.
    Complexity: O(n*m), where n is the length of str1, and m is the length of str2.
    '''
    
    len_x, len_y = len(str1), len(str2) # get length of each string
    matrix = np.zeros((len_x + 1, len_y + 1)) # initiate a matrix of zeros of str1 length x str2 length
    
    # indexing the first row and the first column
    for i in range(1, len_x + 1):
        matrix[i, 0] = i
    
    for j in range(1, len_y + 1):
        matrix[0, j] = j
    
    # compare letter by letter both row-wise and column-wise
    for i in range(1, len_x + 1):
        for j in range(1, len_y + 1):
            if str1[i - 1] == str2[j - 1]:
                substitution_cost = 0
            else:
                substitution_cost = 1
                
            matrix[i, j] = min(matrix[i - 1, j] + 1,                   # insertion
                             matrix[i, j - 1] + 1,                     # deletion
                             matrix[i - 1, j - 1] + substitution_cost) # substitution
    
    return matrix[len_x, len_y]

In [16]:
results = [n for n in unique_names if get_distance(n, target) == 1]

In [17]:
results

['Lucy',
 'Luce',
 'Lua',
 'Lula',
 'Yuca',
 'Lucas',
 'Luba',
 'Lucia',
 'Lupa',
 'Luma',
 'Cuca',
 'Luna']